In [1]:
with open("datasets\libai.txt", 'r', encoding='utf-8') as file:
    content = file.read()

In [2]:
chars = list(set(content))
n_chars = len(chars)
# 字符与对应的数字标记
char_indices = dict((c, i) for i, c in enumerate(chars))
# 数字标记与对应的字符
indices_char = dict((i, c) for i, c in enumerate(chars))

In [3]:
maxlen = 20
step = 3
sentences = []
next_chars = []
for i in range(0, len(content) - maxlen, step):
    sentences.append(content[i: i + maxlen])
    next_chars.append(content[i + maxlen])

In [4]:
# 将所有句子中的字转化为独热编码的形式
import numpy as np
X_train = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y_train = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for j, char in enumerate(sentence):
        X_train[i, j, char_indices[char]] = 1
    y_train[i, char_indices[next_chars[i]]] = 1

In [5]:
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.optimizers import RMSprop
model = Sequential()
model.add(LSTM(units=128, 
               input_shape=(maxlen, n_chars)))
model.add(Dense(units=n_chars, 
                activation='softmax'))
model.summary()

Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128)               1826816   
_________________________________________________________________
dense_1 (Dense)              (None, 3439)              443631    
Total params: 2,270,447
Trainable params: 2,270,447
Non-trainable params: 0
_________________________________________________________________


In [6]:
model.compile(loss='categorical_crossentropy', 
              optimizer=RMSprop(lr=0.01),
              metrics=None)
model.fit(X_train, 
          y_train,
          batch_size=128,
          epochs=50)

Instructions for updating:
Use tf.cast instead.
Epoch 1/50
30727/30727 [==============================] - 48s 2ms/step - loss: 6.5990
Epoch 2/50
30727/30727 [==============================] - 47s 2ms/step - loss: 5.9755
Epoch 3/50
30727/30727 [==============================] - 43s 1ms/step - loss: 5.6476
Epoch 4/50
30727/30727 [==============================] - 42s 1ms/step - loss: 5.2171
Epoch 5/50
30727/30727 [==============================] - 40s 1ms/step - loss: 4.7536
Epoch 6/50
30727/30727 [==============================] - 41s 1ms/step - loss: 4.3145
Epoch 7/50
30727/30727 [==============================] - 40s 1ms/step - loss: 3.9023
Epoch 8/50
30727/30727 [==============================] - 39s 1ms/step - loss: 3.5086
Epoch 9/50
30727/30727 [==============================] - 39s 1ms/step - loss: 3.1767
Epoch 10/50
30727/30727 [==============================] - 40s 1ms/step - loss: 2.8912
Epoch 11/50
30727/30727 [==============================] - 39s 1ms/step - loss: 2.6368
Epoc

In [7]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(n=1, pvals=preds, size=1)
    return np.argmax(probas)

In [12]:
import random
def generate_text(length, diversity):
    # 随机选择一个句子
    start_index = random.randint(0, len(content) - maxlen - 1)
    sentence = content[start_index: start_index + maxlen]
    # 用于存储生成的字
    generated = ''
    for i in range(length):
        x_pred = np.zeros((1, maxlen, len(chars)))
        # 将随机生成的句子中的字转化为独热编码的形式
        for i, char in enumerate(sentence):
            x_pred[0, i, char_indices[char]] = 1
        # 根据随机选择的句子生成生成下一个字
        preds = model.predict(x_pred, verbose=0)[0]
        # 获得生成的字的数字标记
        next_index = sample(preds, diversity)
        # 根据数据数字标记获得对应字符
        next_char = indices_char[next_index]
        # 存储生成的字
        generated += next_char
        # 将生成的字追加随机选择的句子中
        sentence = sentence[1:] + next_char
    return generated

In [13]:
print(generate_text(24, 0.2))

君不仙寒上，语随白萝长。传君玉云难，归风未江月。
